[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/00-langchain-intro.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/00-langchain-intro.ipynb)

#### [LangChain Handbook](https://github.com/pinecone-io/examples/tree/master/generation/langchain/handbook)

# Intro to LangChain

LangChain is a popular framework that allow users to quickly build apps and pipelines around **L**arge **L**anguage **M**odels. It can be used to for chatbots, **G**enerative **Q**uestion-**A**nwering (GQA), summarization, and much more.

The core idea of the library is that we can _"chain"_ together different components to create more advanced use-cases around LLMs. Chains may consist of multiple components from several modules:

* **Prompt templates**: Prompt templates are, well, templates for different types of prompts. Like "chatbot" style templates, ELI5 question-answering, etc

* **LLMs**: Large language models like GPT-3, BLOOM, etc

* **Agents**: Agents use LLMs to decide what actions should be taken, tools like web search or calculators can be used, and all packaged into logical loop of operations.

* **Memory**: Short-term memory, long-term memory.

In [4]:
# !pip install -qU langchain

# Using LLMs in LangChain

LangChain supports several LLM providers, like Hugging Face and OpenAI.

Let's start our exploration of LangChain by learning how to use a few of these different LLM integrations.

## Hugging Face

We first need to install additional prerequisite libraries:

In [5]:
# !pip install -qU huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 5.3 MB/s eta 0:00:00


For Hugging Face models we need a Hugging Face Hub API token. We can find this by first getting an account at [HuggingFace.co](https://huggingface.co/) and clicking on our profile in the top-right corner > click *Settings* > click *Access Tokens* > click *New Token* > set *Role* to *write* > *Generate* > copy and paste the token below:

In [1]:
import os
import local_tokens


HF_API_KEY = local_tokens.api_keys()["HF"]["INIT_READ"]

os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_API_KEY

We can then generate text using a HF Hub model (we'll use `google/flan-t5-x1`) using the Inference API built into Hugging Face Hub.

_(The default Inference API doesn't use specialized hardware and so can be slow and cannot run larger models like `bigscience/bloom-560m` or `google/flan-t5-xxl`)_

In [2]:
import os
import local_tokens


HF_API_KEY = local_tokens.api_keys()["HF"]["INIT_READ"]

os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_API_KEY

from langchain import PromptTemplate, HuggingFaceHub, LLMChain

# initialize HF LLM
flan_t5 = HuggingFaceHub(
	repo_id="google/flan-t5-xxl", 
	model_kwargs={"temperature":0.5, "max_length":32} # too low temperature can take up a lot of time.
)

# build prompt template for simple question-answering
template = """Question: {question}

Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(
    prompt=prompt,
    llm=flan_t5
)

question = "Which NFL team won the Super Bowl in the 2010 season?"

print("Obtaining output")
output = llm_chain.run(question)

print(output)

c:\Users\adity\anaconda3\envs\LC\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


Obtaining output
philadelphia eagles


If we'd like to ask multiple questions we can by passing a list of dictionary objects, where the dictionaries must contain the input variable set in our prompt template (`"question"`) that is mapped to the question we'd like to ask.

In [3]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
res = llm_chain.generate(qs)
res

LLMResult(generations=[[Generation(text='philadelphia eagles')], [Generation(text='192 cm')], [Generation(text='john glenn')], [Generation(text='two')]], llm_output=None, run=[RunInfo(run_id=UUID('f2341729-d35c-4148-a745-6863518ebff4')), RunInfo(run_id=UUID('9585311a-6550-4414-8c0f-dd3f590713f1')), RunInfo(run_id=UUID('ca8c3295-8424-4a2f-af4c-784802b4763e')), RunInfo(run_id=UUID('3e3eb49a-fe84-4ae8-9ecb-8e04626a6793'))])

It is a LLM, so we can try feeding in all questions at once:

In [4]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=flan_t5
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

output = llm_chain.run(qs_str)
print(output)

Atlanta Falcons, 169 centimeters, Buzz Aldrin, Neil Armstrong, and Michael Collins, 20 eyes, 63 centimeters, Buzz Aldrin, Neil Armstrong, and Michael Collins, 121 centimeters, Buzz Aldrin, Neil Armstrong, and Michael Collins, 122 centimeters, Buzz Aldrin, Neil Armstrong, and Michael Collins, 121 centimeters


In [5]:
output = output.split(", ")
for o in output: print(o)

Atlanta Falcons
169 centimeters
Buzz Aldrin
Neil Armstrong
and Michael Collins
20 eyes
63 centimeters
Buzz Aldrin
Neil Armstrong
and Michael Collins
121 centimeters
Buzz Aldrin
Neil Armstrong
and Michael Collins
122 centimeters
Buzz Aldrin
Neil Armstrong
and Michael Collins
121 centimeters


But with this model it doesn't work too well, we'll see this approach works better with different models soon.

## OpenAI

Start by installing additional prerequisites:

In [6]:
# !pip install -qU openai

We can also use OpenAI's generative models. The process is similar, we need to
give our API key which can be retrieved by signing up for an account on the
[OpenAI website](https://openai.com/api/) (see top-right of page). We then pass the API key below:

In [7]:
import os
import local_tokens

os.environ['OPENAI_API_KEY'] = local_tokens.api_keys()["OAI"]["CMN"]

If using OpenAI via Azure you should also set:

```python
os.environ['OPENAI_API_TYPE'] = 'azure'
# API version to use (Azure has several)
os.environ['OPENAI_API_VERSION'] = '2022-12-01'
# base URL for your Azure OpenAI resource
os.environ['OPENAI_API_BASE'] = 'https://your-resource-name.openai.azure.com'
```

Then we decide on which model we'd like to use, there are several options but we will go with `text-davinci-003`:

In [9]:
from langchain.llms import OpenAI

davinci = OpenAI(model_name='text-davinci-003')

Alternatively if using Azure OpenAI we do:

```python
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
    deployment_name="your-azure-deployment", 
    model_name="text-davinci-003"
)
```

We'll use the same simple question-answer prompt template as before with the Hugging Face example. The only change is that we now pass our OpenAI LLM `davinci`:

In [10]:
llm_chain = LLMChain(
    prompt=prompt,
    llm=davinci
)

print(f"Question: {question}")
print(llm_chain.run(question))

Question: Which NFL team won the Super Bowl in the 2010 season?
 The Green Bay Packers won Super Bowl XLV in the 2010 season.


The same works again for multiple questions using `generate`:

In [11]:
qs = [
    {'question': "Which NFL team won the Super Bowl in the 2010 season?"},
    {'question': "If I am 6 ft 4 inches, how tall am I in centimeters?"},
    {'question': "Who was the 12th person on the moon?"},
    {'question': "How many eyes does a blade of grass have?"}
]
llm_chain.generate(qs)

LLMResult(generations=[[Generation(text=' The Green Bay Packers won the Super Bowl in the 2010 season.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' 193.04 centimeters', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' Charles "Pete" Conrad was the 12th person on the moon. He was part of the Apollo 12 mission and was the third person to walk on the moon.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text=' A blade of grass does not have any eyes.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'completion_tokens': 61, 'total_tokens': 136, 'prompt_tokens': 75}, 'model_name': 'text-davinci-003'}, run=[RunInfo(run_id=UUID('4d25b103-f4a8-4122-923b-e24bc0fa0fa4')), RunInfo(run_id=UUID('b902ff07-58ac-43a0-a60c-35fc9dd51c6f')), RunInfo(run_id=UUID('2fef40f5-0923-44af-8380-b6c4b6fd2400')), RunInfo(run_id=UUID('0fa3292f-8314-4949-bc5e-606a8796aa82')

Note that the below format doesn't feed the questions in iteratively but instead all in one chunk.

In [13]:
qs = [
    "Which NFL team won the Super Bowl in the 2010 season?",
    "If I am 6 ft 4 inches, how tall am I in centimeters?",
    "Who was the 12th person on the moon?",
    "How many eyes does a blade of grass have?"
]
print(llm_chain.run(qs))


1. The New Orleans Saints 
2. 193 centimeters 
3. Harrison Schmitt 
4. A blade of grass does not have eyes.


Now we can try to answer all question in one go, as mentioned, more powerful LLMs like `text-davinci-003` will be more likely to handle these more complex queries.

In [14]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(
    template=multi_template,
    input_variables=["questions"]
)

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=davinci
)

qs_str = (
    "Which NFL team won the Super Bowl in the 2010 season?\n" +
    "If I am 6 ft 4 inches, how tall am I in centimeters?\n" +
    "Who was the 12th person on the moon?" +
    "How many eyes does a blade of grass have?"
)

print(llm_chain.run(qs_str))

The New Orleans Saints won the Super Bowl in the 2010 season.
6 ft 4 inches is 193.04 centimeters.
The 12th person on the moon was Harrison Schmitt.
A blade of grass has no eyes.


---